## Lab 3

## Let's try out 3 MCP Servers

We will add new powers to our agents:  
1. Memory
2. Internet search
3. Live market data

This is where you can really appreciate the benefits of MCP: it makes it so easy to equip our agents with tools that others have developed.

In [1]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
import os
from IPython.display import Markdown, display
from datetime import datetime
load_dotenv(override=True)

True

## First new MCP Server: a persistent knowledge-graph based memory

Here's a really interesting one: a knowledge-graph based memory.

It's a persistent memory store of entities, observations about them, and relationships between them.


In [2]:
params = {
  "command": "npx",
  "args": ["-y", "mcp-memory-libsql"],
  "env": {
				"LIBSQL_URL": "file:./memory/ed.db"
			}
}


async with MCPServerStdio(params=params) as server:
    mcp_tools = await server.list_tools()

mcp_tools

[Tool(name='create_entities', description='Create new entities with observations and optional embeddings', inputSchema={'type': 'object', 'properties': {'entities': {'type': 'array', 'items': {'type': 'object', 'properties': {'name': {'type': 'string'}, 'entityType': {'type': 'string'}, 'observations': {'type': 'array', 'items': {'type': 'string'}}, 'embedding': {'type': 'array', 'items': {'type': 'number'}, 'description': 'Optional vector embedding for similarity search'}}, 'required': ['name', 'entityType', 'observations']}}}, 'required': ['entities']}, annotations=None),
 Tool(name='search_nodes', description='Search for entities and their relations using text or vector similarity', inputSchema={'type': 'object', 'properties': {'query': {'oneOf': [{'type': 'string', 'description': 'Text search query'}, {'type': 'array', 'items': {'type': 'number'}, 'description': 'Vector for similarity search'}]}}, 'required': ['query']}, annotations=None),
 Tool(name='read_graph', description='Get 

In [6]:
instructions = "You use your entity tools as a persistent memory to store and recall information about your conversations."
request = "My name's Ed. I'm running a workshop live about AI Agents right now, \
and I'm co-presenting with the legendary presenter, Jon, from the SuperDataScience podcast."
model = "gpt-4o-mini"

In [7]:
async with MCPServerStdio(params=params) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("conversation"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

I've noted that your name is Ed and that you're running a live workshop on AI Agents, co-presenting with Jon from the SuperDataScience podcast. If you need any specific information or assistance during your workshop, feel free to ask!

In [10]:
question = "My name's Ed. What do you know about me?"

async with MCPServerStdio(params=params) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("conversation"):
        result = await Runner.run(agent, question)
    display(Markdown(result.final_output))

I know that your name is Ed. You've been involved in running a workshop live about AI Agents and co-presenting with Jon from the SuperDataScience podcast. If there's anything else you'd like to share or discuss, feel free!

### Check the trace:

https://platform.openai.com/traces

## Second new MCP Server: Brave Search



https://brave.com/search/api/

Set up your account, and put your key in the .env under `BRAVE_API_KEY`

In [11]:
env = {"BRAVE_API_KEY": os.getenv("BRAVE_API_KEY")}
params = {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-brave-search"], "env": env}

async with MCPServerStdio(params=params) as server:
    mcp_tools = await server.list_tools()

In [12]:
mcp_tools

[Tool(name='brave_web_search', description='Performs a web search using the Brave Search API, ideal for general queries, news, articles, and online content. Use this for broad information gathering, recent events, or when you need diverse web sources. Supports pagination, content filtering, and freshness controls. Maximum 20 results per request, with offset for pagination. ', inputSchema={'type': 'object', 'properties': {'query': {'type': 'string', 'description': 'Search query (max 400 chars, 50 words)'}, 'count': {'type': 'number', 'description': 'Number of results (1-20, default 10)', 'default': 10}, 'offset': {'type': 'number', 'description': 'Pagination offset (max 9, default 0)', 'default': 0}}, 'required': ['query']}, annotations=None),
 Tool(name='brave_local_search', description="Searches for local businesses and places using Brave's Local Search API. Best for queries related to physical locations, businesses, restaurants, services, etc. Returns detailed information including:\

In [13]:
instructions = "You are able to search the web for information and briefly summarize the takeaways."
request = f"Please research the latest news on Amazon stock price and briefly summarize its outlook. \
For context, the current date is {datetime.now().strftime('%Y-%m-%d')}"
model = "gpt-4o-mini"

In [14]:
async with MCPServerStdio(params=params) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("conversation"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

As of May 11, 2025, here’s a summary of the latest news and outlook regarding Amazon's stock price:

1. **Recent Performance**: Amazon's stock (AMZN) has faced significant challenges in 2025, currently down approximately 14.63% year-to-date. In May alone, the stock dropped about 11.43%, reflecting a rough month for investors.

2. **Earnings and Indicators**: The company recently reported strong growth in its advertising division, generating $13.9 billion with a 19% year-over-year increase. Additionally, Amazon Web Services (AWS) experienced a 17% year-over-year growth. However, these positives have not yet translated into a recovery in stock price.

3. **Market Sentiment**: Analysts are debating whether to buy the dip or to avoid the stock, with mixed feelings about its future. There are concerns over economic factors, including tariffs introduced by the Trump administration, which may affect operational costs and margins.

4. **Future Predictions**: Forecasts suggest that Amazon's stock could average around $207 in May, with potential volatility leading to a maximum price of $243 and a minimum of $178.

Overall, while Amazon shows growth in key operational areas, its stock performance in 2025 has been underwhelming, causing concern among investors about its near-term outlook.

## Third MCP Server: Polygon.io MCP Server

Introducing polygon.io

Polygon.io is a hugely popular financial data provider. It has a free plan and a paid plan. And it also has an MCP Server!

First, read up on polygon.io on their excellent website, including looking at their pricing:

https://polygon.io

1. Please sign up for polygon.io (top right)  
2. Once signed in, please select "Keys" in the left hand navigation
3. Press the blue "New Key" button
4. Copy the key name
5. Edit your .env file and add the row:

`POLYGON_API_KEY=xxxx`

This section covers the paid plan, but I've set up a free alternative too. Just use the `.env` file to control which one is used:

If you do decide to have a paid plan, please add this to your .env file to indicate:

`POLYGON_PLAN=paid`

And if you decide to go all the way for the realtime API, then please do:

`POLYGON_PLAN=realtime`

In [15]:
polygon_api_key = os.getenv("POLYGON_API_KEY")

params = {"command": "uvx",
          "args": ["--from", "git+https://github.com/polygon-io/mcp_polygon@master", "mcp_polygon"],
          "env": {"POLYGON_API_KEY": polygon_api_key}
          }
async with MCPServerStdio(params=params) as server:
    mcp_tools = await server.list_tools()
mcp_tools


[Tool(name='get_aggs', description='\nList aggregate bars for a ticker over a given date range in custom time window sizes.\n', inputSchema={'properties': {'ticker': {'title': 'Ticker', 'type': 'string'}, 'multiplier': {'title': 'Multiplier', 'type': 'integer'}, 'timespan': {'title': 'Timespan', 'type': 'string'}, 'from_': {'anyOf': [{'type': 'string'}, {'type': 'integer'}, {'format': 'date-time', 'type': 'string'}, {'format': 'date', 'type': 'string'}], 'title': 'From'}, 'to': {'anyOf': [{'type': 'string'}, {'type': 'integer'}, {'format': 'date-time', 'type': 'string'}, {'format': 'date', 'type': 'string'}], 'title': 'To'}, 'adjusted': {'anyOf': [{'type': 'boolean'}, {'type': 'null'}], 'default': None, 'title': 'Adjusted'}, 'sort': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': None, 'title': 'Sort'}, 'limit': {'anyOf': [{'type': 'integer'}, {'type': 'null'}], 'default': None, 'title': 'Limit'}, 'params': {'anyOf': [{'additionalProperties': True, 'type': 'object'}, {'typ

### Wow that's a lot of tools!

Let's try them out - hopefully the sheer number of tools doesn't overwhelm gpt-4o-mini!

With the $29 monthly plan, we don't have access to some of the APIs, so I've needed to specify which APIs can be called.

If you've splashed out on a bigger plan, feel free to remove my extra constraint..

In [16]:
instructions = "You answer questions about the stock market."
request = "What's the share price of Apple? Use your get_snapshot_ticker tool to get the latest price."
model = "gpt-4o-mini"

async with MCPServerStdio(params=params) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("conversation"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

The latest share price of Apple (AAPL) is **$198.53**. Here are some additional details:

- **Today's Change**: +$1.03 (+0.52%)
- **Opening Price**: $199.00
- **Day's High**: $200.54
- **Day's Low**: $197.54
- **Volume**: 36,452,015 shares

If you need more information, feel free to ask!

## In total, how many MCP servers and tools have we looked at?

In [17]:
from mcp_params import trader_mcp_server_params, researcher_mcp_server_params

all_params = trader_mcp_server_params + researcher_mcp_server_params("ed")

count = 0
for each_params in all_params:
    async with MCPServerStdio(params=each_params) as server:
        mcp_tools = await server.list_tools()
        count += len(mcp_tools)
print(f"We have {len(all_params)} MCP servers, and {count} tools")

We have 6 MCP servers, and 44 tools


## OK - Let's build an application that uses all of these tools!!